# Extract Airbnb Listings per Nbh required
----

#### Note
* Dataset request from rapidApi/Mashvisor endpoint short term rentals/listings

In [1]:
# Interactive plots
%matplotlib notebook

# Dependencies and Setup
import requests
import random
import json
import ast
import os
import datetime
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import scipy.stats as st
#from citipy import citipy
#from pprint import pprint

# Set by selecting a datasource import type (CSV or API)
source = 'api'
#source = 'csv'

# Remember to update the config file with your API key
from api_keys import mash_api_key

# Set variable for current date using datetime library
today = datetime.date.today()

# Set Mashvisor API URL for rental rates
url = "https://mashvisor-api.p.rapidapi.com/airbnb-property/active-listings"

# Set todays date as active and formatted as mm/dd/yy for API import
date = today.strftime('%m/%d/%y')
datef = today.strftime('%m%d%y')

# Set a specific date as active parameter for CSV import
#date = '09/12/20'
#datef = '091220'

# Print active date
print(date)
print(datef)
#print(mash_api_key)

12/09/20
120920


In [2]:
# Retrieve cities required
nbhReq = []
f = open(f'../data/mash-api/{datef}/top_nbh_id-{datef}j.json', 'r')
s = f.read()
#s = s.replace("\'", "\"")
s = s.replace("null", "''")
nbhstr = json.dumps(s)
nbhjson = json.loads(nbhstr)
nbhbnb = ast.literal_eval(nbhjson)
nbhReq.append(nbhbnb)
df_nbh = pd.DataFrame(nbhReq)
df_nbh = df_nbh.T
df_nbh.head()

,0
273471,"{'name': 'Dignowity Hill-St. Paul Square', 'ci..."
271352,"{'name': 'Downtown', 'city': 'San Antonio', 's..."
275804,"{'name': 'United Homeowners', 'city': 'San Ant..."
271349,"{'name': 'Dignowity Hill', 'city': 'San Antoni..."
274033,"{'name': 'Historic Gardens', 'city': 'San Anto..."


In [3]:
# Create and call Mashvisor API query string for airbnb listings
properties = []
for i in range(len(df_nbh)):
    querystring1 = {"state":f'{df_nbh[0][i]["state"]}',"neighborhood":f'{df_nbh.index[i]}', "items":20, "page":1}
    querystring2 = {"state":f'{df_nbh[0][i]["state"]}',"neighborhood":f'{df_nbh.index[i]}', "items":20, "page":2}
    querystring3 = {"state":f'{df_nbh[0][i]["state"]}',"neighborhood":f'{df_nbh.index[i]}', "items":20, "page":3}

    #print(querystring1,querystring2,querystring3)
    print(f'extracting latest airbnb listings in neighborhood {df_nbh.index[i]}...')
    headers = {
        'x-rapidapi-key': mash_api_key,
        'x-rapidapi-host': "mashvisor-api.p.rapidapi.com"
    }
    response1 = requests.request("GET", url, headers=headers, params=querystring1)
    print('1 data retrieved')
    response2 = requests.request("GET", url, headers=headers, params=querystring2)
    print('2 data retrieved')
    response3 = requests.request("GET", url, headers=headers, params=querystring3)
    print('3 data retrieved')
    
    properties.append(response1.json())
    properties.append(response2.json())
    properties.append(response3.json())
    
    print('data merged')
    print('')

extracting latest airbnb listings in neighborhood 273471...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 271352...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 275804...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 271349...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 274033...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 271360...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 271303...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 271325...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracti

1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 806851...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 764088...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 124368...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 417159...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 417189...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 417154...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 417161...
1 data retrieved
2 data retrieved
3 data retrieved
data merged

extracting latest airbnb listings in neighborhood 417160...
1 data r

In [9]:
print(f'total records requested and retrieved - {len(properties)}')
df_props = pd.DataFrame(properties)
df_props.head()

total records requested and retrieved - 315


,status,content
0,success,"{'num_of_properties': 216, 'items': 20, 'total..."
1,success,"{'num_of_properties': 216, 'items': 20, 'total..."
2,success,"{'num_of_properties': 216, 'items': 20, 'total..."
3,success,"{'num_of_properties': 214, 'items': 20, 'total..."
4,success,"{'num_of_properties': 214, 'items': 20, 'total..."


In [10]:
reqProperties = []
for i in range(len(df_props)):
    for j in range(len(properties[i]['content']['properties'])):
        
        if (properties[i]['content']['properties'][j]['cleaning_fee_native'] is not None):
            cleaning_fee = int(properties[i]['content']['properties'][j]['cleaning_fee_native'])
        else:
            cleaning_fee = 0

        if (properties[i]['content']['properties'][j]['num_of_rooms'] is not None):
            room_count = int(properties[i]['content']['properties'][j]['num_of_rooms'])
        else:
            room_count = 0

        if (properties[i]['content']['properties'][j]['num_of_beds'] is not None):
            bed_count = int(properties[i]['content']['properties'][j]['num_of_beds'])
        else:
            bed_count = 0

        if (properties[i]['content']['properties'][j]['capacity_of_people'] is not None):
            max_capacity = int(properties[i]['content']['properties'][j]['capacity_of_people'])
        else:
            max_capacity = 0
        
        if (properties[i]['content']['properties'][j]['start_rating'] is not None):
            star_rating = int(properties[i]['content']['properties'][j]['start_rating'])
        else:
            star_rating = 0

        if (properties[i]['content']['properties'][j]['reviews_count'] is not None):
            total_reviews = int(properties[i]['content']['properties'][j]['reviews_count'])
        else:
            total_reviews = 0

        if (properties[i]['content']['properties'][j]['nights_booked'] > 0):
            airbnbListing = {
                "airbnb_id":int(properties[i]['content']['properties'][j]['id']),
                "host_id": int(properties[i]['content']['properties'][j]['property_id']),
                "nbh_id": int(properties[i]['content']['properties'][j]['airbnb_neighborhood_id']),
                "lat": float(properties[i]['content']['properties'][j]['lat']),
                "lon": float(properties[i]['content']['properties'][j]['lon']),
                'city':properties[i]['content']['properties'][j]['airbnb_city'],
                'state':properties[i]['content']['properties'][j]['state'],
                "night_price":float(properties[i]['content']['properties'][j]['night_price']),
                "cleaning_fee": cleaning_fee,
                "nights_booked":int(properties[i]['content']['properties'][j]['nights_booked']),
                "rental_income": int(properties[i]['content']['properties'][j]['rental_income']),
                "property_type": properties[i]['content']['properties'][j]['property_type'],
                "room_count": room_count,
                "bed_count": bed_count,
                "max_capacity": max_capacity,
                "star_rating":star_rating,
                "total_reviews": total_reviews,
                "date_created":properties[i]['content']['properties'][j]['created_at'],
                "date_updated":properties[i]['content']['properties'][j]['updated_at']
            }
            reqProperties.append(airbnbListing)
            
            
df_reqProp = pd.DataFrame(reqProperties)
df_reqProp
df_reqProp.index = df_reqProp['airbnb_id']
del df_reqProp['airbnb_id']
df_reqProp

,host_id,nbh_id,lat,lon,city,state,night_price,cleaning_fee,nights_booked,rental_income,property_type,room_count,bed_count,max_capacity,star_rating,total_reviews,date_created,date_updated
airbnb_id,,,,,,,,,,,,,,,,,,
27874948,11555442,273471,29.4175,-98.4960,San Antonio,TX,305.0,175,59,1800,House,5,3,8,5,8,2020-10-30T17:58:18.000Z,2020-10-30T17:58:18.000Z
27874733,11654041,273471,29.4165,-98.4858,San Antonio,TX,159.0,70,107,1547,Guesthouse,1,1,3,5,55,2020-10-30T17:57:51.000Z,2020-10-30T17:57:51.000Z
27874722,13361985,273471,29.4273,-98.4893,San Antonio,TX,119.0,25,93,922,Apartment,1,1,2,5,103,2020-10-30T17:57:50.000Z,2020-10-30T17:57:50.000Z
27874807,13882643,273471,29.4107,-98.4776,San Antonio,TX,54.0,75,201,905,Guesthouse,1,2,4,5,28,2020-10-30T17:58:02.000Z,2020-10-30T17:58:02.000Z
27874771,13996317,273471,29.4133,-98.4953,San Antonio,TX,104.0,50,243,2106,Apartment,1,1,2,5,190,2020-10-30T17:57:57.000Z,2020-10-30T17:57:57.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23700606,1667080,403492,38.9043,-77.0002,Washington,DC,99.0,50,329,2714,Guest suite,0,1,2,5,213,2020-10-31T02:30:20.000Z,2020-10-31T02:30:20.000Z
23700478,16882568,403492,38.8847,-77.0035,Washington,DC,112.0,40,313,2921,Apartment,1,2,5,5,284,2020-10-31T02:29:44.000Z,2020-10-31T02:29:44.000Z
23699872,17156975,403492,38.8965,-76.9991,Washington,DC,152.0,135,265,3357,Apartment,2,2,6,5,216,2020-10-31T02:27:03.000Z,2020-10-31T02:27:03.000Z


In [11]:
result = df_reqProp.to_json(orient="index")
parsed = json.loads(result)
airbnbProps_json = json.dumps(parsed, indent=4)
#print(airbnbProps_json)

In [21]:
#split dataframe records to file export 
#df_sp1 = df_reqProp[0:1000]
#df_sp2 = df_reqProp[1001:2000]
#df_sp3 = df_reqProp[2001:3000]
#df_sp4 = df_reqProp[3001:4000]
#df_sp5 = df_reqProp[4001:5000]
#df_sp6 = df_reqProp[5001:6049]

#list_df_sp = [df_sp1,df_sp2,df_sp3,df_sp4,df_sp5,df_sp6]
#airbnbProps_json_sp = []
#for i in range(len(list_df_sp)):
#    result = list_df_sp[i].to_json(orient="index")
#    parsed = json.loads(result)
#    airbnbProps_json_sp.append(json.dumps(parsed, indent=4))

#print(len(airbnbProps_json_sp))

6


In [12]:
# all in one file export
with open(f'../data/mash-api/{datef}/airbnb-listings-{datef}.json', 'w') as f1:
    json.dump(airbnbProps_json,f1,indent=4)
    print("The airbnb listings json file is created")
f1.close()

The airbnb listings json file is created


In [22]:
# split file export
#for i in range(len(airbnbProps_json_sp)):
#    with open(f'../data/mash-api/{datef}/airbnb-listings-{datef}-{i}.json', 'w') as f2:
#        json.dump(airbnbProps_json_sp[i],f2,indent=4)
#        print(f'The airbnb split listings json file {i} is created')
#    f2.close()

The airbnb split listings json file 0 is created
The airbnb split listings json file 1 is created
The airbnb split listings json file 2 is created
The airbnb split listings json file 3 is created
The airbnb split listings json file 4 is created
The airbnb split listings json file 5 is created


In [8]:
# END JSON EXTRACT FROM API